In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


In [ ]:
from os import environ
from sqlalchemy import create_engine

%load_ext dotenv
%dotenv

POSTGRES_ADDRESS = environ['DB_HOST']
POSTGRES_PORT = environ['DB_PORT']
POSTGRES_USERNAME = environ['DB_USER']
POSTGRES_PASSWORD = environ['DB_PASSWORD']
POSTGRES_DBNAME = environ['DB_NAME']

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

connection = create_engine(postgres_str)


In [ ]:
checkins = pd.read_sql_query(
    '''SELECT * FROM checkins;''',
    connection,
)

reservations = pd.read_sql_query(
    '''SELECT * FROM reservations;''',
    connection,
)

users = pd.read_sql_query(
    '''SELECT * FROM users WHERE role = 'user' ;''',
    connection,
)


In [ ]:
reservations_x_checkin = pd.merge(
    reservations,
    checkins,
    how='inner',
    left_on=['user_id', 'reservation_date'],
    right_on=['user_id', 'checkin_date'],
)


In [ ]:
def get_frequency(user):
    reservations_made = reservations['user_id'][reservations['user_id'] == user.id].count(
    )
    reservations_with_checkins = reservations_x_checkin['user_id'][reservations_x_checkin['user_id'] == user.id].count(
    )

    return round((reservations_with_checkins / reservations_made) * 100, 2)


users['frequency'] = users.apply(lambda row: get_frequency(row), axis=1)


In [ ]:
output_users = users[['first_name', 'last_name',
                      'email', 'frequency']].sort_values('frequency')

figure, ax = plt.subplots(figsize=(12, 12))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=output_users.values,
                 colLabels=output_users.columns, loc='center')

pp = PdfPages("output.pdf")
pp.savefig(figure, bbox_inches='tight')
pp.close()
